# Event labelling using structures. Case: BiographySampo

In [512]:
import pandas as pd

Current structures used for event detection: dictionary of structures

structures = {'name of the event':[list of the structures corresponding to the event]}

Type of event structures:
- one verb (tuple)
- verb + adverb (tuple)
- verb + noun (tuple with a dict for noun: {'noun': morphological features}

For now, additional arguments, such as e.g. to whom the person is married, are not taken into account.

In [424]:
df = pd.read_csv('text_corpus.csv', encoding='utf-8', sep=',')

#4 events with their most frequent structures and keywords stored in the dictionary 
structures = {'avioliitto':[('solmia',{'avioliitto':'Case=Gen'}),('avioitua',),('mennä',{'naimisiin':''}),],
              'kuolema':[('kuolla',),('hukkua',),('menehtyä',),('tehdä',{'itsemurha':'Case=Gen'}),],
              'muuttaminen':[('muuttaa',),('lähteä',),('siirtyä',),],
              'tutkinnon suorittaminen':[('suorittaa',{'tutkinto':'Case=Gen'}),('valmistua',),('tulla',{'maisteri':'Case=Tra'}),('päästä',{'maisteri':'Case=Tra'}),],
             }

df.head()

,id,structure,word,originalForm,wID,sID,sentence,paragraph,pID,paragraphText,upos,feat,head,lemma,wordOrderNumber,sentenceOrderNumber,paragraphOrderNumber
0,http://ldf.fi/nbf/p5450,http://ldf.fi/nbf/biography/s2355,http://ldf.fi/nbf/biography/p37614#s1.1,Harrastuksiltaan,1,1.0,http://ldf.fi/nbf/biography/p37614#s1.0,http://ldf.fi/nbf/biography/p37614,8,Harrastuksiltaan ja perhesuhteiltaan fysiikan ...,NOUN,Case=Abl|Number=Plur|Person[psor]=3,http://ldf.fi/nbf/biography/p37614#s1.6,harrastus,1,1.0,8
1,http://ldf.fi/nbf/p5450,http://ldf.fi/nbf/biography/s2355,http://ldf.fi/nbf/biography/p37614#s1.2,ja,2,1.0,http://ldf.fi/nbf/biography/p37614#s1.0,http://ldf.fi/nbf/biography/p37614,8,Harrastuksiltaan ja perhesuhteiltaan fysiikan ...,CONJ,NaN,http://ldf.fi/nbf/biography/p37614#s1.1,ja,2,1.0,8
2,http://ldf.fi/nbf/p5450,http://ldf.fi/nbf/biography/s2355,http://ldf.fi/nbf/biography/p37614#s1.3,perhesuhteiltaan,3,1.0,http://ldf.fi/nbf/biography/p37614#s1.0,http://ldf.fi/nbf/biography/p37614,8,Harrastuksiltaan ja perhesuhteiltaan fysiikan ...,NOUN,Case=Abl|Number=Plur|Person[psor]=3,http://ldf.fi/nbf/biography/p37614#s1.1,perhesuhteet,3,1.0,8
3,http://ldf.fi/nbf/p5450,http://ldf.fi/nbf/biography/s2355,http://ldf.fi/nbf/biography/p37614#s1.4,fysiikan,4,1.0,http://ldf.fi/nbf/biography/p37614#s1.0,http://ldf.fi/nbf/biography/p37614,8,Harrastuksiltaan ja perhesuhteiltaan fysiikan ...,NOUN,Case=Gen|Number=Sing,http://ldf.fi/nbf/biography/p37614#s1.5,fysiikka,4,1.0,8
4,http://ldf.fi/nbf/p5450,http://ldf.fi/nbf/biography/s2355,http://ldf.fi/nbf/biography/p37614#s1.5,professori,5,1.0,http://ldf.fi/nbf/biography/p37614#s1.0,http://ldf.fi/nbf/biography/p37614,8,Harrastuksiltaan ja perhesuhteiltaan fysiikan ...,NOUN,Case=Nom|Number=Sing,http://ldf.fi/nbf/biography/p37614#s1.6,professori,5,1.0,8


In [425]:
data = df.to_dict()
biographies = data['id'] #currently not used, but may be useful in the future
lemmas = data['lemma']
paragraphs = data['paragraph']
features = data['feat']
sentences = data['sentence']

In [440]:
#function for printing data for debugging purposes and for storing the results to dictionaries
def update_data(event, key_begin, key_end, dicts, labels):
    for row in range(key_begin,key_end+1):
        print('For {0}: {1} in the paragraph {2} in the sentence {3}'.format(event, lemmas[row], paragraphs[row],sentences[row]))
    dicts[event].append({biographies[row]:{paragraphs[row]:sentences[row]}})
    if labels[sentences[row]] == [None]:
        labels.update({sentences[row]:[event]})
    else:
        labels[sentences[row]].append(event)

In [516]:
dicts = {'avioliitto':[], 'kuolema':[],'muuttaminen':[], 'tutkinnon suorittaminen':[]}
labels = dict(zip(sentences.values(), [[None]]*len(sentences)))

In [517]:
#iterating over all lemmas
for key, value in lemmas.items():
    #iterating over all structures
    for event, keywords in structures.items():
        for j in keywords:
            #if it is verb + adverb/noun
            if len(j) == 2:
                if value == j[0]:
                    print(key)
                    key_begin = key_end = key
                    if type(j[1]) is dict:
                        print('is dict')
                        
                        while sentences[key_begin] == sentences[key_end]:
                            k = list(j[1].keys())[0]
                            v = list(j[1].values())[0]
                            
                            #finding the end of the structure
                            #true if v is an empty string in case there are no particular morphological features
                            if lemmas[key_end] == k and v in features[key_end]:
                                print(lemmas[key_end])
                                print(key_end)
                                update_data(event, key_begin, key_end, dicts, labels)
                                break
                            else:
                                key_end += 1
                    else:                            
                        if lemmas[key_end] == j[1]:
                            print(key_end)
                            print('no morphology')
                            update_data(event, key_begin, key_end, dicts, labels)
                            break
                        else:
                            key_end += 1
            #if the structure is a verb only
            else:
                if value == j[0]:
                    key_begin = key_end = key
                    print(key_begin)
                    print('not dict')
                    print('now printing with function')
                    update_data(event, key_begin, key_end, dicts, labels)   

15
is dict
16
not dict
now printing with function
For avioliitto: avioitua in the paragraph http://ldf.fi/nbf/biography/p37614 in the sentence http://ldf.fi/nbf/biography/p37614#s2.0
29
is dict
avioliitto
31
For avioliitto: solmia in the paragraph http://ldf.fi/nbf/biography/p27849 in the sentence http://ldf.fi/nbf/biography/p27849#s1.0
For avioliitto: 26vuotias in the paragraph http://ldf.fi/nbf/biography/p27849 in the sentence http://ldf.fi/nbf/biography/p27849#s1.0
For avioliitto: avioliitto in the paragraph http://ldf.fi/nbf/biography/p27849 in the sentence http://ldf.fi/nbf/biography/p27849#s1.0
67
is dict
avioliitto
68
For avioliitto: solmia in the paragraph http://ldf.fi/nbf/biography/p45772 in the sentence http://ldf.fi/nbf/biography/p45772#s1.0
For avioliitto: avioliitto in the paragraph http://ldf.fi/nbf/biography/p45772 in the sentence http://ldf.fi/nbf/biography/p45772#s1.0
100
is dict
180
not dict
now printing with function
For avioliitto: avioitua in the paragraph http://

In [519]:
def remove_none(dicts):
    clean_dict = dicts.copy()
    for key in dicts.keys():
        if dicts[key] == [None]:
            del clean_dict[key]
    return clean_dict
labels

{'http://ldf.fi/nbf/biography/p37614#s1.0': [None],
 'http://ldf.fi/nbf/biography/p37614#s2.0': ['avioliitto'],
 'http://ldf.fi/nbf/biography/p27849#s1.0': ['avioliitto'],
 'http://ldf.fi/nbf/biography/p27849#s2.0': [None],
 'http://ldf.fi/nbf/biography/p45772#s1.0': ['avioliitto'],
 'http://ldf.fi/nbf/biography/p45772#s2.0': [None],
 'http://ldf.fi/nbf/biography/p157661#s1.0': [None],
 'http://ldf.fi/nbf/biography/p157661#s2.0': [None],
 'http://ldf.fi/nbf/biography/p157661#s3.0': [None],
 'http://ldf.fi/nbf/biography/p137729#s1.0': ['avioliitto'],
 'http://ldf.fi/nbf/biography/p137729#s2.0': [None],
 'http://ldf.fi/nbf/biography/p137729#s3.0': [None],
 'http://ldf.fi/nbf/biography/p24913#s1.0': [None],
 'http://ldf.fi/nbf/biography/p24913#s2.0': [None],
 'http://ldf.fi/nbf/biography/p24913#s3.0': [None],
 'http://ldf.fi/nbf/biography/p24913#s4.0': [None],
 'http://ldf.fi/nbf/biography/p24913#s5.0': [None],
 'http://ldf.fi/nbf/biography/p24913#s6.0': ['avioliitto'],
 'http://ldf.fi/nb

In [535]:
clean_labels = remove_none(labels)
results = pd.DataFrame({'label':list(clean_labels.values()),'sentence': list(clean_labels.keys())})
results.label.value_counts()

[kuolema]                                                                       40
[muuttaminen]                                                                   32
[avioliitto]                                                                    20
[tutkinnon suorittaminen]                                                        7
[tutkinnon suorittaminen, tutkinnon suorittaminen]                               6
[muuttaminen, muuttaminen]                                                       5
[kuolema, kuolema]                                                               2
[tutkinnon suorittaminen, muuttaminen, tutkinnon suorittaminen, muuttaminen]     1
[avioliitto, kuolema]                                                            1
[kuolema, avioliitto]                                                            1
[muuttaminen, kuolema]                                                           1
Name: label, dtype: int64

In [549]:
test_labels = pd.read_csv('test_data_labels.csv', encoding='utf-8', sep=';')
test_labels.label.value_counts()

kuolema                    25
avioliitto                 25
muuttaminen                25
tutkinnon suorittaminen    25
Name: label, dtype: int64

In [545]:
results_dict = results.to_dict()
#n = list(results_dict['label'].values()).index(['tutkinnon suorittaminen', 'muuttaminen', 'tutkinnon suorittaminen', 'muuttaminen'])
n = list(results_dict['label'].values()).index(['kuolema', 'kuolema'])
sent = results_dict['sentence'][n]
parag = sent.split('#')[0]
print('Sentence {0} in paragraph {1}'.format(sent, parag))
print(list(results_dict['label'].values()))

Sentence http://ldf.fi/nbf/biography/p114399#s1.0 in paragraph http://ldf.fi/nbf/biography/p114399
[['avioliitto'], ['avioliitto'], ['avioliitto'], ['avioliitto'], ['avioliitto'], ['kuolema'], ['avioliitto'], ['kuolema'], ['avioliitto'], ['avioliitto'], ['kuolema'], ['avioliitto'], ['kuolema'], ['avioliitto', 'kuolema'], ['kuolema'], ['kuolema'], ['avioliitto'], ['kuolema'], ['muuttaminen'], ['avioliitto'], ['kuolema'], ['avioliitto'], ['muuttaminen'], ['kuolema'], ['avioliitto'], ['avioliitto'], ['kuolema'], ['avioliitto'], ['kuolema', 'avioliitto'], ['avioliitto'], ['avioliitto'], ['muuttaminen'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema', 'kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['avioliitto'], ['muuttaminen', 'kuolema'], ['kuolema'], ['muuttaminen'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema'], ['kuolema

In [546]:
s = list(sentences.values()).index(sent)
df.loc[s,]

id                                                 http://ldf.fi/nbf/p593
structure                              http://ldf.fi/nbf/biography/s10101
word                             http://ldf.fi/nbf/biography/p114399#s1.1
originalForm                                                       Nystén
wID                                                                     1
sID                                                                     1
sentence                         http://ldf.fi/nbf/biography/p114399#s1.0
paragraph                             http://ldf.fi/nbf/biography/p114399
pID                                                                    15
paragraphText           Nystén kuoli 60-vuotiaana vilustuttuaan suojel...
upos                                                                 NOUN
feat                                                 Case=Nom|Number=Sing
head                             http://ldf.fi/nbf/biography/p114399#s1.2
lemma                                 

In [544]:
df.paragraphText.loc[s,]

'Nystén kuoli 60-vuotiaana vilustuttuaan suojeluskuntaonnettomuudessa menehtyneen HOP:n virkailijan hautaussaattueessa. Nystén oli luonut maan kolmanneksi suurimman liikepankin, joka oli tuolloin melko vakaalla pohjalla. Liittopankin saneeraaminen oli tosin edellyttänyt HOP:n osakepääoman puolittamista, ja tervehdyttämiskustannuksia riitti monelle vuodelle. Rasitukset vaikuttivat siihen, ettei HOP:n talletuskanta 1930-luvulla juuri noussut, ja se menetti jatkuvasti markkinaosuutta etenkin paikallispankeille. Nysténin kauden päättyessä HOP oli silti elinkykyinen ja Suomen oloissa merkittävä rahalaitos.'